<a href="https://colab.research.google.com/github/Rishabhc711/Rishabhc711/blob/master/ImageCaptioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt',delimiter='\t',header=None)

In [ ]:
print(dataset.shape)

(40460, 2)


In [ ]:
dataset.head()

,0,1
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


Cleaning The Text

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, dataset.shape[0]):
    cap = re.sub('[^a-zA-Z]', ' ', dataset[1][i])
    cap = cap.lower()
    cap = cap.split()
    cap=[word for word in cap if len(word)>1]
    cap = ' '.join(cap)
    cap= '<startseq> '+cap+' <endseq>'
    corpus.append(cap) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
print(corpus[:50])
print(len(corpus))

['<startseq> child in pink dress is climbing up set of stairs in an entry way <endseq>', '<startseq> girl going into wooden building <endseq>', '<startseq> little girl climbing into wooden playhouse <endseq>', '<startseq> little girl climbing the stairs to her playhouse <endseq>', '<startseq> little girl in pink dress going into wooden cabin <endseq>', '<startseq> black dog and spotted dog are fighting <endseq>', '<startseq> black dog and tri colored dog playing with each other on the road <endseq>', '<startseq> black dog and white dog with brown spots are staring at each other in the street <endseq>', '<startseq> two dogs of different breeds looking at each other on the road <endseq>', '<startseq> two dogs on pavement moving toward each other <endseq>', '<startseq> little girl covered in paint sits in front of painted rainbow with her hands in bowl <endseq>', '<startseq> little girl is sitting in front of large painted rainbow <endseq>', '<startseq> small girl in the grass plays with 

In [ ]:
for i in range(dataset.shape[0]):
  dataset[0][i]=dataset[0][i].split(".")[0]

In [ ]:
print(len(dataset))

40460


In [ ]:
new_dict={}
i=0
for i in range(int(len(corpus)/5)):
  new_dict[dataset[0][5*i]]=[(caption) for caption in corpus[i:i+5]]

In [ ]:
print(len(new_dict))

8092


In [ ]:
print(new_dict['2513260012_03d33305cf'])

['<startseq> man with orange hat and brown jacket has cigarette in his mouth <endseq>', '<startseq> there is man with brown hat lighting cigarette <endseq>', '<startseq> fisherman is reeling his rod while another relaxes in boat on water <endseq>', '<startseq> man in gray cap fishing <endseq>', '<startseq> man sits in boat and holds fishing pole while another lays down <endseq>']


In [ ]:
all_vocab = []

for key in new_dict.keys():
    [ all_vocab.append(i) for des in new_dict[key] for i in des.split()]

In [ ]:
print("total words appearing : " , len(all_vocab))

total words appearing :  457923


In [ ]:
print(all_vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from collections import Counter

counter = Counter(all_vocab)

In [ ]:
dic_ = dict(counter)

In [ ]:
sorted_dic = sorted(dic_.items(), key = lambda x: x[1], reverse=True)

In [ ]:
print(len(sorted_dic))

4019


In [ ]:
threshold_value = 10

d = [(x) for x in sorted_dic if x[1]>threshold_value]

In [ ]:
len(d)

1729

In [ ]:
all_vocab = [x[0] for x in d]

In [ ]:
len(all_vocab)

1729

In [ ]:
with open('/content/drive/My Drive/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt') as f:
    train = f.read()

In [ ]:
train = [e[:-4] for e in train.split('\n')[:-1]]

In [ ]:
print(len(train))

6000


In [ ]:
with open('/content/drive/My Drive/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt') as f:
    test = f.read()

In [ ]:
test = [e[:-4] for e in test.split('\n')[:-1]]

In [ ]:
train_descriptions = {}
for t in train:
    train_descriptions[t] = []
    for cap in new_dict[t]:
        train_descriptions[t].append(cap)

In [ ]:
len(train_descriptions)

6000

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.preprocessing import image
import time
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *

In [ ]:
images=('/content/drive/My Drive/Flickr_Data/Flickr_Data/Images/')

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [ ]:
model = ResNet50(weights = 'imagenet', input_shape = (224,224,3))

102973440/102967424 [==============================] - 2s 0us/step


In [ ]:
model_new = Model(inputs = model.input, outputs =  model.layers[-2].output)

In [ ]:
def preprocess_image(img):
    img = image.load_img(img, target_size=(224,224))
    img = image.img_to_array(img)
    #print(img.shape)
    img = preprocess_input(img)
    #print(img.shape)
    img = np.expand_dims(img, axis = 0)
    #print(img.shape)

    return img

In [ ]:
def encode_image(img):
    img = preprocess_image(img)
    fea_vec = model_new.predict(img)
    #print(fea_vec.shape)
    fea_vec = fea_vec.reshape(fea_vec.shape[1],1 )
    #print(fea_vec.shape)
    return fea_vec

In [ ]:
import os
print(len(os.listdir(images)))

8091


In [ ]:
print(images)
img = images+train[0]+".jpg"
print(img[len(images):])

/content/drive/My Drive/Flickr_Data/Flickr_Data/Images/
2513260012_03d33305cf.jpg


In [ ]:
start = time.time()

encoding_train = {}

for ix, img in enumerate(train):
    
    img = images+train[ix]+".jpg"
    
    p = encode_image(img) 
    encoding_train[ img[len(images):] ] = p 
    
    
    if ix%100 == 0:
        print("Encoding image :" + str(ix))
    
print("Time taken in sec - " + time.time() - start)

Encoding image :0
Encoding image :100
Encoding image :200
Encoding image :300
Encoding image :400
Encoding image :500
Encoding image :600
Encoding image :700
Encoding image :800
Encoding image :900
Encoding image :1000
Encoding image :1100
Encoding image :1200
Encoding image :1300


KeyboardInterrupt: ignored

In [ ]:
def data_generator(train_descriptions, encoding_train, word_to_idx, max_len,  num_photos_per_batch ):
    X1, X2, y = [], [], []
    
    n=0
    
    while True:
        
        for key, desc_list in train_descriptions.items():
            n+=1
            
            #photo = encoding_train[key+'.jpg']
            
            for desc in desc_list:
                
                seq = [word_to_idx[word] for word in desc.split() if word in word_to_idx]
                
                
                for i in range(1, len(seq)):
                    in_seq = seq[0:i]
                    out_seq = seq[i] 
                    
                    in_seq = pad_sequences( [in_seq], maxlen=max_len, value= 0, padding='post')[0]
                
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0] 
                    
                    #X1.append(photo)
                    X1.append(in_seq)
                    X2.append(in_seq)
                    y.append(out_seq)
                    
            if n == num_photos_per_batch:
                yield [[np.array(X1), np.array(X2)] , np.array(y) ]
                X1, X2, y = [], [], []
                n =0

In [ ]:
for i in data_generator(train_descriptions, encoding_train, word_to_idx, max_len, 3):
    X, y = i
    print(X[0].shape)
    print(X[1].shape)
    print(y.shape)
    break

(151, 2048, 1)
(151, 35)
(151, 4058)


In [ ]:
word_to_idx = {}
idx_to_word = {}

ix = 1

for e in all_vocab:
    word_to_idx[e] = ix
    idx_to_word[ix] = e
    
    ix +=1

In [ ]:
all_caption_len = []

for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        all_caption_len.append(len(cap.split()))

In [ ]:
max_len = max(all_caption_len)
print(max_len)

35


In [ ]:
vocab_size = len(idx_to_word) + 1
print(vocab_size) 

4058


In [ ]:
print(len(all_vocab))

1729
